In [1]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
import os
from langsmith import traceable
import json
import asyncio
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
AZURE_OPENAI_API_KEY=os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT=os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_API_VERSION=os.environ["AZURE_OPENAI_API_VERSION"]

In [3]:
kernel = Kernel()

service = AzureChatCompletion(
    deployment_name="gpt-4o-mini",
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION
)

kernel.add_service(service)

In [4]:
from semantic_kernel.functions import kernel_function

class MathTools:

    @kernel_function(name="calculate", description="Perform math calculation")
    def calculate(self, expression: str) -> str:
        try:
            return str(eval(expression))
        except Exception:
            return "Invalid expression"


kernel.add_plugin(MathTools(), plugin_name="math_tools")
settings_tools = AzureChatPromptExecutionSettings(
    tool_call_behavior="auto"
)

In [ ]:
agent_prompt = """
You are an AI assistant.
If the question requires calculation, call the tool.
Otherwise answer directly.

Question: {{$input}}
"""

agent_function = kernel.add_function(
    plugin_name="tool_agent",
    function_name="decider",
    prompt=agent_prompt,
    prompt_execution_settings=settings_tools
)

In [6]:
from langsmith import traceable

@traceable(name="Tool Agent Step")
async def run_tool_agent(q):
    return await kernel.invoke(agent_function, input=q)


In [7]:
@traceable(name="Tool-Using Agent")
async def tool_using_agent(user_question):

    response = await run_tool_agent(user_question)
    return response


In [8]:
answer = await tool_using_agent("What is 245 * 82?")

In [9]:
print(answer)

The result of 245 * 82 is 20,090.
